#SETUP

Downloading files to be sent:

In [1]:
!ls

1.1.1.jpg
1.1.2.jpg
1.1.3.jpg
1.1.4.jpg
1.1.jpg
1.2.jpg
1.3.jpg
1.4.jpg
1.5.jpg
1.jpg
2.jpg
big_text_file.txt
big_text_file.txt.1
bla_bla_bla
CC451 - Computer Networks - Spring 2018 - Sheets.pdf
ch2 summary.pdf
Chapter_1_V6.1.pdf
Chapter_1_V6.1.ppt
Chapter_2_V6.3.pdf
Chapter_2_V6.3.ppt
Chapter_3_V6.01.pdf
Chapter_3_V6.0.ppt
Chapter_4_V6.2.ppt
Chapter_4_V7.01.pdf
Chapter_5_V6.01.ppt
Chapter_6_V6.0.ppt
Chapter_7_V6.0.ppt
Chapter_8_V6.0.ppt
Chapter_9_V6.0.ppt
down_test_file_7.txt
drive-download-20180504T080804Z-001.zip
Networks_project.ipynb
Networks-sheet2-kareem-3356.odt
Networks Sheet 4.odt
Networks Sheet 4.pdf
notes and questions
Podington_Bear_-_Starling.mp3
project.zip
receiver_stats.txt
recv.py
sender.py
sender_stats.txt
Sheet_1.pdf
Sheet_2.pdf
sheet 3.odt
sheet 3.pdf
Sheet_4.pdf
sheet 5- kareem -3356.odt
sheet 5- kareem -3356.pdf
Sheet_5.pdf
Socket_Prog.ipynb
Socket_Programming2.ipynb
Socket_Programming.ipynb
SSPNetworksSpring_2018Introduction.ppt
stop_and_wait.ipynb
test_file.txt

In [2]:
#!wget https://www.dropbox.com/s/4fm193mtun5nzl0/big_text_file.txt  # big_text_file.txt file of 4.00 MB that will be sent
#!wget https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt      # small test text file
#!wget https://freemusicarchive.org/file/music/Music_for_Video/Podington_Bear/Solo_Instruments/Podington_Bear_-_Starling.mp3  # +4 MB mp3 file

sender and receiver:

In [3]:
import socket, pickle
receiverIP=socket.gethostbyname(socket.gethostname())
from socket import *
senderPort = 12000
receiverPort = 5200 

packets and Acks classes:

In [4]:
class Packet:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  data=None,
  type_ =1,
  act_num=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.data=data
    self.type_=type_
    self.act_num=act_num
    
  def __lt__(self, other):
        return self.seq_num < other.seq_num

class ACK:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  real=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.real=real
  

checksum calculation trial:

In [5]:
#!pip install hashlib
import struct
import hashlib
f = open("test_file.txt", 'rb')
#read one byte in bunary format
data = f.read(1)
print(data)

x=hashlib.sha224(data).hexdigest()
y=hashlib.sha224(data).hexdigest()


print(x == y)

b'k'
True


randomly dropping packets function:

In [6]:
import random

def decision(probability):
    return random.random() < probability

Dividing files to packets and vice versa:

and writing logs:

In [7]:
import csv

def file_to_pkt(file_name, sw, seq_num_range):
    packets = []
    f = open(file_name, 'rb')
    data = f.read(1024)
    seqNO = 0
    while data:
        if sw: #make sequence numbers 0 and 1 only in stop and wait
            packets.append(Packet(seq_num=seqNO % 2, checksum=hashlib.sha224(data).hexdigest(), data=data,act_num=seqNO))

        else:
            packets.append(Packet(seq_num=seqNO % seq_num_range, checksum=hashlib.sha224(data).hexdigest(), data=data,act_num=seqNO))
        data = f.read(1024)
        
        seqNO += 1
    #type 0 is the terminating packet of the file   
    packets[len(packets) - 1].type_ = 0
    return packets
  
  

def pkt_to_file(file_name, packets):
    f = open(file_name, 'wb')
    for i in range(len(packets)):
        f.write(packets[i].data)
    f.close()
    
def get_stats(file_name,stats_dict):
  with open(file_name, "w") as f:
    writer = csv.writer(f, delimiter="\t", lineterminator="\r\n")
    writer.writerows(stats_dict.items())
    


#STOP AND WAIT

In [8]:
#@title Parameters Tuning { form-width: "30%"}
sent_file_name = "Podington_Bear_-_Starling.mp3" #@param {type:"string"}
recv_file_name = "stop&wait.mp3" #@param {type:"string"}
prob = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
variable_name = 0
prob_check=1

import os
import time


start_time = time.time()

  
def recv_child():
    stats_ = {}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    
    print ('--The receiver is ready to receive--\n')

    packets = []
    actual_seq_num = 0
    expected_seq_num = 0
    while True:
        (message, senderAddress) = receiverSocket.recvfrom(4096)
        #if a packet is received
        message = pickle.loads(message)
        #if checksum is correct
        if(decision(prob_check)):
                            
          if(message.checksum == hashlib.sha224(message.data).hexdigest()):
                    
            t6 = time.time()

            if expected_seq_num == message.seq_num:
                packets.append(message)
          
                if decision(prob):
                    receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num,
                            message.checksum, actual_seq_num,message.data)), senderAddress)
                    t6 = time.time()
                    stats_['{0:.6f}'.format(t6 - start_time)] = \
                        'receiver sent ack for packet ' \
                        + str(expected_seq_num) + ' actual: ' \
                        + str(actual_seq_num)
                    actual_seq_num += 1
                    expected_seq_num = actual_seq_num % 2

                    if message.type_ == 0:
                        time.sleep(0.1)

                        print ('--Whole file received successfully. File is ready to download--\n')

                        get_stats('receiver_stats.txt', stats_)

                        pkt_to_file(recv_file_name, packets)
                    else:

                        stats_['{0:.6f}'.format(t6 - start_time)] = \
                            'receiver dropped ack for packet ' \
                            + str(expected_seq_num) + ' actual: ' \
                            + str(actual_seq_num)
                        actual_seq_num += 1
                        expected_seq_num = actual_seq_num % 2
                else:

                    # duplicate

                    receiverSocket.sendto(pickle.dumps(ACK(message.seq_num, message.checksum,
                                          actual_seq_num - 1,message.data)), senderAddress)

                    stats_['{0:.6f}'.format(t6 - start_time)] = \
                        '--duplicate-- already received ' \
                        + str(message.seq_num) + ' actual: ' \
                        + str(actual_seq_num - 1)
          else:
            print("checksum is really wrong xD")
            print("expected : ",message.checksum)
            print("received : ",hashlib.sha224(message.data).hexdigest() )
        else:
          print("simulated wrong checksum detected for packet : ",message.seq_num)
          print("expected : ",message.checksum)
          print("received : ",hashlib.sha224(message.data).hexdigest() )
      

def send_parent():
  
    newpid = os.fork()
    if newpid == 0:
        recv_child()
    else:
        stats_ = {}
        time.sleep(3)
        pids = (os.getpid(), newpid)
        packets = []

        packets = file_to_pkt(file_name=sent_file_name, sw=True,  seq_num_range= 0)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))
        message = packets

        for i in range(len(packets)):
            if decision(prob):
                senderSocket.sendto(pickle.dumps(message[i]),
                                    (receiverIP, receiverPort))
                #print("sending message of data :",message[i].data)
                #print("sending message of checksum :",message[i].checksum)
                #print("check should be ",hashlib.sha224(message[i].data).hexdigest())
                t1 = time.time()
                stats_['{0:.6f}'.format(t1 - start_time)] = 'sent packet ' \
                    + str(i % 2) + ' actual: ' + str(i)
            else:
                t1 = time.time()
                stats_['{0:.6f}'.format(t1 - start_time)] = 'sent packet ' \
                    + str(i % 2) + ' actual: ' + str(i)

            while True:

                modifiedMessage = pot_ack = None
                try:
                    senderSocket.settimeout(0.01)
                    (modifiedMessage, receiverAddress) = \
                        senderSocket.recvfrom(4096)
                    #receive a packet
                    t5 = time.time()
                    pot_ack = pickle.loads(modifiedMessage)
                    #checking checksum
                    if(pot_ack.checksum == hashlib.sha224(pot_ack.data).hexdigest() ):
                      
                      if isinstance(pot_ack, ACK):
                          if pot_ack.seq_num == i % 2:
                              stats_['{0:.6f}'.format(t5 - start_time)] = \
                                  'sender received ack for packet ' \
                                  + str(pot_ack.seq_num) + ' actual: ' \
                                  + str(pot_ack.real)
                              modifiedMessage = None

                              if i == len(packets) - 1:
                                  get_stats('sender_stats.txt', stats_)
                                  print ('--Whole file sent successfully--\n')

                          break
                          
                      else:
                        print("wrong checksum detected for Ack : ",pot_ack.real,pot_ack.seq_num)
                        print("expected : ",pot_ack.checksum)
                        print("received : ",hashlib.sha224(pot_ack.data).hexdigest() )
                        
                except:
                    senderSocket.settimeout(None)
                    t8 = time.time()
                    stats_['{0:.6f}'.format(t8 - start_time)] = \
                        '--timeout-- no ack received for packet ' \
                        + str(i % 2) + ' actual: ' + str(i)

                    if decision(prob):
                        senderSocket.sendto(pickle.dumps(message[i]),
                                (receiverIP, receiverPort))
                        t7 = time.time()
                        stats_['{0:.6f}'.format(t7 - start_time)] = \
                            '--again-- sent packet ' + str(i % 2) \
                            + ' actual: ' + str(i)
                    else:
                        t7 = time.time()
                        stats_['{0:.6f}'.format(t7 - start_time)] = \
                            '--again-- dropped packet ' + str(i % 2) \
                            + ' actual: ' + str(i)

    senderSocket.close()


send_parent()


--The receiver is ready to receive--



TypeError: __init__() takes from 1 to 4 positional arguments but 5 were given

--Whole file sent successfully--



In [9]:
!ls

1.1.1.jpg
1.1.2.jpg
1.1.3.jpg
1.1.4.jpg
1.1.jpg
1.2.jpg
1.3.jpg
1.4.jpg
1.5.jpg
1.jpg
2.jpg
big_text_file.txt
big_text_file.txt.1
bla_bla_bla
CC451 - Computer Networks - Spring 2018 - Sheets.pdf
ch2 summary.pdf
Chapter_1_V6.1.pdf
Chapter_1_V6.1.ppt
Chapter_2_V6.3.pdf
Chapter_2_V6.3.ppt
Chapter_3_V6.01.pdf
Chapter_3_V6.0.ppt
Chapter_4_V6.2.ppt
Chapter_4_V7.01.pdf
Chapter_5_V6.01.ppt
Chapter_6_V6.0.ppt
Chapter_7_V6.0.ppt
Chapter_8_V6.0.ppt
Chapter_9_V6.0.ppt
down_test_file_7.txt
drive-download-20180504T080804Z-001.zip
Networks_project.ipynb
Networks-sheet2-kareem-3356.odt
Networks Sheet 4.odt
Networks Sheet 4.pdf
notes and questions
Podington_Bear_-_Starling.mp3
project.zip
receiver_stats.txt
recv.py
sender.py
sender_stats.txt
Sheet_1.pdf
Sheet_2.pdf
sheet 3.odt
sheet 3.pdf
Sheet_4.pdf
sheet 5- kareem -3356.odt
sheet 5- kareem -3356.pdf
Sheet_5.pdf
Socket_Prog.ipynb
Socket_Programming2.ipynb
Socket_Programming.ipynb
SSPNetworksSpring_2018Introduction.ppt
stop_and_wait.ipynb
test_file.txt

In [ ]:
#!ps -fA | grep python
from google.colab import files
files.download(recv_file_name)
files.download('receiver_stats.txt')
files.download('sender_stats.txt')

#GBN

In [10]:
from threading import Thread, Event, Timer, Lock
import time

def TimerReset(*args, **kwargs):
    """ Global function for Timer """
    return _TimerReset(*args, **kwargs)


class _TimerReset(Thread):

    def __init__(self, interval, function, args=[], kwargs={}):
        Thread.__init__(self)
        self.interval = interval
        self.function = function
        self.args = args
        self.kwargs = kwargs
        self.finished = Event()
        self.resetted = True

    def cancel(self):
       
        self.finished.set()

    def run(self):
        while not self.finished.isSet():
            
    
            self.resetted = True
            while self.resetted:
                
                self.resetted = False
                self.finished.wait(self.interval)
    
            if not self.finished.isSet():
                self.function(*self.args, **self.kwargs)
        

    def reset(self, interval=None):
        """ Reset the timer """

        if interval:
            
            self.interval = interval
        else:
            
            pass

        self.resetted = True
        self.finished.set()
        self.finished.clear()

In [11]:
lock = Lock()

def resend(Packet,prob,start_time,stats_,senderSocket):

    if decision(prob):
      with lock:
        senderSocket.sendto(pickle.dumps(Packet),(receiverIP, receiverPort))
        t7 = time.time()
        stats_['{0:.6f}'.format(t7 - start_time)] = \
        'sent packet ' + str(Packet.seq_num) \
        + ' actual: ' + str(Packet.act_num)
        print(stats_['{0:.6f}'.format(t7 - start_time)])

    else:
      t7 = time.time()
      stats_['{0:.6f}'.format(t7 - start_time)] = \
      'dropped packet ' + str(Packet.seq_num) \
      + ' actual: ' + str(Packet.act_num)

    
    return 
        

In [13]:
#@title Parameters Tuning { form-width: "40%"}
sent_file_name = "Podington_Bear_-_Starling.mp3" #@param {type:"string"}
recv_file_name = "gbn.mp3" #@param {type:"string"}
prob = 1 #@param {type:"slider", min:0, max:1, step:0.01}
seq_num_range = 8 #@param {type:"integer"}
window_size = 4 #@param {type:"integer"}
time_out = 0.05 #@param {type:"number"}

import heapq
import os
import time


start_time = time.time()


def recv_child():
    stats_ = {}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    
    print ('--The receiver is ready to receive--\n')

    packets = []
    actual_seq_num = 0
    expected_seq_num = 0
    while True:

        (message, senderAddress) = receiverSocket.recvfrom(1024*(window_size+1))
        message = pickle.loads(message)
        t6 = time.time()
        
        if expected_seq_num == message.seq_num:
            packets.append(message)
            if decision(prob):
                receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num,
                        0, actual_seq_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver sent ack for packet ' \
                    + str(expected_seq_num) + ' actual: ' \
                    + str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % seq_num_range

                if message.type_ == 0:
                    time.sleep(0.1)
                    
                    print ('--Whole file received successfully. File is ready to download--\n')

                    get_stats('receiver_stats.txt', stats_)

                    pkt_to_file(recv_file_name, packets)
            else:

                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver dropped ack for packet ' \
                    + str(expected_seq_num) + ' actual: ' \
                    + str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % seq_num_range
        else:

            # duplicate

            receiverSocket.sendto(pickle.dumps(ACK(message.seq_num, 0,
                                  actual_seq_num - 1)), senderAddress)

            stats_['{0:.6f}'.format(t6 - start_time)] = \
                '--out_of_order-- received ' \
                + str(message.seq_num) + ' expected: ' \
                + str(expected_seq_num) 



#Sender
def send_parent():
    #sender process
    newpid = os.fork()
    #if in child
    if newpid == 0:
        recv_child()
    else:#parent
        stats_ = {}
        time.sleep(1)
        packets = file_to_pkt(file_name=sent_file_name, sw=False,
                              seq_num_range=seq_num_range)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))

        # ## initial setting of window

        base = 0
        index = 0
        step = 0
        window = []
        tm=None
        not_acked = []
        acked_num = 0

        if index + window_size < len(packets):
            window = packets[index:(index + window_size)]
        else:
            window = packets[index:len(packets)]

        for i in range(len(window)):
            #for each packet in window create an event object and  a timer/thread
            #key of these dics to get these object is packet sequence number
            senderSocket.sendto(pickle.dumps(window[i]),(receiverIP, receiverPort))
           

            not_acked.append(window[i].seq_num)
        
        tm= TimerReset(time_out,resend,[window,prob,start_time,stats_,senderSocket])
        tm.start()
        while True:

            reply = None
            with lock:
              (reply, receiverAddress) = senderSocket.recvfrom(4096)
          
            reply = pickle.loads(reply)
            print("ack seq_num:",reply.seq_num,reply.real,"REC. ACK")
            if isinstance(reply,ACK):
               
                

                if (reply.seq_num in not_acked):
                    
                    not_acked.remove(reply.seq_num)

                    acked_num += 1

                    if acked_num == len(packets):
                        tm.cancel()
                        senderSocket.close()
                        break

                    if base == reply.seq_num:
                        
                        if len(not_acked)==0:
                            step=window_size
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) > window_size - 1:
                            step = abs(seq_num_range % base - heapq.nsmallest(1, not_acked)[0])
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) <= window_size - 1:
                            step = abs(base - heapq.nsmallest(1, not_acked)[0])

                        index+=step

                        if index + window_size < len(packets):
                            
                            window = packets[index:index + window_size]
 
                        else:

                            window = packets[index:len(packets)]
                                    
                        

                        base= window[0].seq_num
                        print(window[0].act_num," ---- ",window[len(window)-1].act_num)
                        
                        tm.reset()
                
                        for item in window[-step:]:
                            senderSocket.sendto(pickle.dumps(item),(receiverIP, receiverPort))
                            not_acked.append(item.seq_num)



        

        return



send_parent()


--The receiver is ready to receive--

ack seq_num: 0 0 REC. ACK
1  ----  4
ack seq_num: 1 1 REC. ACK
2  ----  5
ack seq_num: 2 2 REC. ACK
3  ----  6
ack seq_num: 3 3 REC. ACK
4  ----  7
ack seq_num: 4 4 REC. ACK
5  ----  8
ack seq_num: 5 5 REC. ACK
8  ----  11
ack seq_num: 6 6 REC. ACK
ack seq_num: 7 7 REC. ACK
ack seq_num: 0 8 REC. ACK
9  ----  12
ack seq_num: 1 9 REC. ACK
10  ----  13
ack seq_num: 2 10 REC. ACK
11  ----  14
ack seq_num: 3 11 REC. ACK
12  ----  15
ack seq_num: 4 12 REC. ACK
13  ----  16
ack seq_num: 5 13 REC. ACK
16  ----  19
ack seq_num: 6 14 REC. ACK
ack seq_num: 7 15 REC. ACK
ack seq_num: 0 16 REC. ACK
17  ----  20
ack seq_num: 1 17 REC. ACK
18  ----  21
ack seq_num: 2 18 REC. ACK
19  ----  22
ack seq_num: 3 19 REC. ACK
20  ----  23
ack seq_num: 4 20 REC. ACK
21  ----  24
ack seq_num: 5 21 REC. ACK
24  ----  27
ack seq_num: 6 22 REC. ACK
ack seq_num: 7 23 REC. ACK
ack seq_num: 0 24 REC. ACK
25  ----  28
ack seq_num: 1 25 REC. ACK
26  ----  29
ack seq_num: 2 26 REC.

ack seq_num: 1 305 REC. ACK
306  ----  309
ack seq_num: 2 306 REC. ACK
307  ----  310
ack seq_num: 3 307 REC. ACK
308  ----  311
ack seq_num: 4 308 REC. ACK
309  ----  312
ack seq_num: 5 309 REC. ACK
312  ----  315
ack seq_num: 6 310 REC. ACK
ack seq_num: 7 311 REC. ACK
ack seq_num: 0 312 REC. ACK
313  ----  316
ack seq_num: 1 313 REC. ACK
314  ----  317
ack seq_num: 2 314 REC. ACK
315  ----  318
ack seq_num: 3 315 REC. ACK
316  ----  319
ack seq_num: 4 316 REC. ACK
317  ----  320
ack seq_num: 5 317 REC. ACK
320  ----  323
ack seq_num: 6 318 REC. ACK
ack seq_num: 7 319 REC. ACK
ack seq_num: 0 320 REC. ACK
321  ----  324
ack seq_num: 1 321 REC. ACK
322  ----  325
ack seq_num: 2 322 REC. ACK
323  ----  326
ack seq_num: 3 323 REC. ACK
324  ----  327
ack seq_num: 4 324 REC. ACK
325  ----  328
ack seq_num: 5 325 REC. ACK
328  ----  331
ack seq_num: 6 326 REC. ACK
ack seq_num: 7 327 REC. ACK
ack seq_num: 0 328 REC. ACK
329  ----  332
ack seq_num: 1 329 REC. ACK
330  ----  333
ack seq_num: 2 

ack seq_num: 1 545 REC. ACK
546  ----  549
ack seq_num: 2 546 REC. ACK
547  ----  550
ack seq_num: 3 547 REC. ACK
548  ----  551
ack seq_num: 4 548 REC. ACK
549  ----  552
ack seq_num: 5 549 REC. ACK
552  ----  555
ack seq_num: 6 550 REC. ACK
ack seq_num: 7 551 REC. ACK
ack seq_num: 0 552 REC. ACK
553  ----  556
ack seq_num: 1 553 REC. ACK
554  ----  557
ack seq_num: 2 554 REC. ACK
555  ----  558
ack seq_num: 3 555 REC. ACK
556  ----  559
ack seq_num: 4 556 REC. ACK
557  ----  560
ack seq_num: 5 557 REC. ACK
560  ----  563
ack seq_num: 6 558 REC. ACK
ack seq_num: 7 559 REC. ACK
ack seq_num: 0 560 REC. ACK
561  ----  564
ack seq_num: 1 561 REC. ACK
562  ----  565
ack seq_num: 2 562 REC. ACK
563  ----  566
ack seq_num: 3 563 REC. ACK
564  ----  567
ack seq_num: 4 564 REC. ACK
565  ----  568
ack seq_num: 5 565 REC. ACK
568  ----  571
ack seq_num: 6 566 REC. ACK
ack seq_num: 7 567 REC. ACK
ack seq_num: 0 568 REC. ACK
569  ----  572
ack seq_num: 1 569 REC. ACK
570  ----  573
ack seq_num: 2 

ack seq_num: 5 829 REC. ACK
832  ----  835
ack seq_num: 6 830 REC. ACK
ack seq_num: 7 831 REC. ACK
ack seq_num: 0 832 REC. ACK
833  ----  836
ack seq_num: 1 833 REC. ACK
834  ----  837
ack seq_num: 2 834 REC. ACK
835  ----  838
ack seq_num: 3 835 REC. ACK
836  ----  839
ack seq_num: 4 836 REC. ACK
837  ----  840
ack seq_num: 5 837 REC. ACK
840  ----  843
ack seq_num: 6 838 REC. ACK
ack seq_num: 7 839 REC. ACK
ack seq_num: 0 840 REC. ACK
841  ----  844
ack seq_num: 1 841 REC. ACK
842  ----  845
ack seq_num: 2 842 REC. ACK
843  ----  846
ack seq_num: 3 843 REC. ACK
844  ----  847
ack seq_num: 4 844 REC. ACK
845  ----  848
ack seq_num: 5 845 REC. ACK
848  ----  851
ack seq_num: 6 846 REC. ACK
ack seq_num: 7 847 REC. ACK
ack seq_num: 0 848 REC. ACK
849  ----  852
ack seq_num: 1 849 REC. ACK
850  ----  853
ack seq_num: 2 850 REC. ACK
851  ----  854
ack seq_num: 3 851 REC. ACK
852  ----  855
ack seq_num: 4 852 REC. ACK
853  ----  856
ack seq_num: 5 853 REC. ACK
856  ----  859
ack seq_num: 6 

ack seq_num: 1 1113 REC. ACK
1114  ----  1117
ack seq_num: 2 1114 REC. ACK
1115  ----  1118
ack seq_num: 3 1115 REC. ACK
1116  ----  1119
ack seq_num: 4 1116 REC. ACK
1117  ----  1120
ack seq_num: 5 1117 REC. ACK
1120  ----  1123
ack seq_num: 6 1118 REC. ACK
ack seq_num: 7 1119 REC. ACK
ack seq_num: 0 1120 REC. ACK
1121  ----  1124
ack seq_num: 1 1121 REC. ACK
1122  ----  1125
ack seq_num: 2 1122 REC. ACK
1123  ----  1126
ack seq_num: 3 1123 REC. ACK
1124  ----  1127
ack seq_num: 4 1124 REC. ACK
1125  ----  1128
ack seq_num: 5 1125 REC. ACK
1128  ----  1131
ack seq_num: 6 1126 REC. ACK
ack seq_num: 7 1127 REC. ACK
ack seq_num: 0 1128 REC. ACK
1129  ----  1132
ack seq_num: 1 1129 REC. ACK
1130  ----  1133
ack seq_num: 2 1130 REC. ACK
1131  ----  1134
ack seq_num: 3 1131 REC. ACK
1132  ----  1135
ack seq_num: 4 1132 REC. ACK
1133  ----  1136
ack seq_num: 5 1133 REC. ACK
1136  ----  1139
ack seq_num: 6 1134 REC. ACK
ack seq_num: 7 1135 REC. ACK
ack seq_num: 0 1136 REC. ACK
1137  ----  114

...

2985  ----  2988
ack seq_num: 1 2985 REC. ACK
2986  ----  2989
ack seq_num: 2 2986 REC. ACK
2987  ----  2990
ack seq_num: 3 2987 REC. ACK
2988  ----  2991
ack seq_num: 4 2988 REC. ACK
2989  ----  2992
ack seq_num: 5 2989 REC. ACK
2992  ----  2995
ack seq_num: 6 2990 REC. ACK
ack seq_num: 7 2991 REC. ACK
ack seq_num: 0 2992 REC. ACK
2993  ----  2996
ack seq_num: 1 2993 REC. ACK
2994  ----  2997
ack seq_num: 2 2994 REC. ACK
2995  ----  2998
ack seq_num: 3 2995 REC. ACK
2996  ----  2999
ack seq_num: 4 2996 REC. ACK
2997  ----  3000
ack seq_num: 5 2997 REC. ACK
3000  ----  3003
ack seq_num: 6 2998 REC. ACK
ack seq_num: 7 2999 REC. ACK
ack seq_num: 0 3000 REC. ACK
3001  ----  3004
ack seq_num: 1 3001 REC. ACK
3002  ----  3005
ack seq_num: 2 3002 REC. ACK
3003  ----  3006
ack seq_num: 3 3003 REC. ACK
3004  ----  3007
ack seq_num: 4 3004 REC. ACK
3005  ----  3008
ack seq_num: 5 3005 REC. ACK
3008  ----  3011
ack seq_num: 6 3006 REC. ACK
ack seq_num: 7 3007 REC. ACK
ack seq_num: 0 3008 REC. AC

3221  ----  3224
ack seq_num: 5 3221 REC. ACK
3224  ----  3227
ack seq_num: 6 3222 REC. ACK
ack seq_num: 7 3223 REC. ACK
ack seq_num: 0 3224 REC. ACK
3225  ----  3228
ack seq_num: 1 3225 REC. ACK
3226  ----  3229
ack seq_num: 2 3226 REC. ACK
3227  ----  3230
ack seq_num: 3 3227 REC. ACK
3228  ----  3231
ack seq_num: 4 3228 REC. ACK
3229  ----  3232
ack seq_num: 5 3229 REC. ACK
3232  ----  3235
ack seq_num: 6 3230 REC. ACK
ack seq_num: 7 3231 REC. ACK
ack seq_num: 0 3232 REC. ACK
3233  ----  3236
ack seq_num: 1 3233 REC. ACK
3234  ----  3237
ack seq_num: 2 3234 REC. ACK
3235  ----  3238
ack seq_num: 3 3235 REC. ACK
3236  ----  3239
ack seq_num: 4 3236 REC. ACK
3237  ----  3240
ack seq_num: 5 3237 REC. ACK
3240  ----  3243
ack seq_num: 6 3238 REC. ACK
ack seq_num: 7 3239 REC. ACK
ack seq_num: 0 3240 REC. ACK
3241  ----  3244
ack seq_num: 1 3241 REC. ACK
3242  ----  3245
ack seq_num: 2 3242 REC. ACK
3243  ----  3246
ack seq_num: 3 3243 REC. ACK
3244  ----  3247
ack seq_num: 4 3244 REC. AC

ack seq_num: 0 3448 REC. ACK
3449  ----  3452
ack seq_num: 1 3449 REC. ACK
3450  ----  3453
ack seq_num: 2 3450 REC. ACK
3451  ----  3454
ack seq_num: 3 3451 REC. ACK
3452  ----  3455
ack seq_num: 4 3452 REC. ACK
3453  ----  3456
ack seq_num: 5 3453 REC. ACK
3456  ----  3459
ack seq_num: 6 3454 REC. ACK
ack seq_num: 7 3455 REC. ACK
ack seq_num: 0 3456 REC. ACK
3457  ----  3460
ack seq_num: 1 3457 REC. ACK
3458  ----  3461
ack seq_num: 2 3458 REC. ACK
3459  ----  3462
ack seq_num: 3 3459 REC. ACK
3460  ----  3463
ack seq_num: 4 3460 REC. ACK
3461  ----  3464
ack seq_num: 5 3461 REC. ACK
3464  ----  3467
ack seq_num: 6 3462 REC. ACK
ack seq_num: 7 3463 REC. ACK
ack seq_num: 0 3464 REC. ACK
3465  ----  3468
ack seq_num: 1 3465 REC. ACK
3466  ----  3469
ack seq_num: 2 3466 REC. ACK
3467  ----  3470
ack seq_num: 3 3467 REC. ACK
3468  ----  3471
ack seq_num: 4 3468 REC. ACK
3469  ----  3472
ack seq_num: 5 3469 REC. ACK
3472  ----  3475
ack seq_num: 6 3470 REC. ACK
ack seq_num: 7 3471 REC. AC

3748  ----  3751
ack seq_num: 4 3748 REC. ACK
3749  ----  3752
ack seq_num: 5 3749 REC. ACK
3752  ----  3755
ack seq_num: 6 3750 REC. ACK
ack seq_num: 7 3751 REC. ACK
ack seq_num: 0 3752 REC. ACK
3753  ----  3756
ack seq_num: 1 3753 REC. ACK
3754  ----  3757
ack seq_num: 2 3754 REC. ACK
3755  ----  3758
ack seq_num: 3 3755 REC. ACK
3756  ----  3759
ack seq_num: 4 3756 REC. ACK
3757  ----  3760
ack seq_num: 5 3757 REC. ACK
3760  ----  3763
ack seq_num: 6 3758 REC. ACK
ack seq_num: 7 3759 REC. ACK
ack seq_num: 0 3760 REC. ACK
3761  ----  3764
ack seq_num: 1 3761 REC. ACK
3762  ----  3765
ack seq_num: 2 3762 REC. ACK
3763  ----  3766
ack seq_num: 3 3763 REC. ACK
3764  ----  3767
ack seq_num: 4 3764 REC. ACK
3765  ----  3768
ack seq_num: 5 3765 REC. ACK
3768  ----  3771
ack seq_num: 6 3766 REC. ACK
ack seq_num: 7 3767 REC. ACK
ack seq_num: 0 3768 REC. ACK
3769  ----  3772
ack seq_num: 1 3769 REC. ACK
3770  ----  3773
ack seq_num: 2 3770 REC. ACK
3771  ----  3774
ack seq_num: 3 3771 REC. AC

ack seq_num: 1 4049 REC. ACK
4050  ----  4053
ack seq_num: 2 4050 REC. ACK
4051  ----  4054
ack seq_num: 3 4051 REC. ACK
4052  ----  4055
ack seq_num: 4 4052 REC. ACK
4053  ----  4056
ack seq_num: 5 4053 REC. ACK
4056  ----  4059
ack seq_num: 6 4054 REC. ACK
ack seq_num: 7 4055 REC. ACK
ack seq_num: 0 4056 REC. ACK
4057  ----  4060
ack seq_num: 1 4057 REC. ACK
4058  ----  4061
ack seq_num: 2 4058 REC. ACK
4059  ----  4062
ack seq_num: 3 4059 REC. ACK
4060  ----  4063
ack seq_num: 4 4060 REC. ACK
4061  ----  4064
ack seq_num: 5 4061 REC. ACK
4064  ----  4067
ack seq_num: 6 4062 REC. ACK
ack seq_num: 7 4063 REC. ACK
ack seq_num: 0 4064 REC. ACK
4065  ----  4068
ack seq_num: 1 4065 REC. ACK
4066  ----  4069
ack seq_num: 2 4066 REC. ACK
4067  ----  4070
ack seq_num: 3 4067 REC. ACK
4068  ----  4071
ack seq_num: 4 4068 REC. ACK
4069  ----  4072
ack seq_num: 5 4069 REC. ACK
4072  ----  4075
ack seq_num: 6 4070 REC. ACK
ack seq_num: 7 4071 REC. ACK
ack seq_num: 0 4072 REC. ACK
4073  ----  407

In [ ]:
from google.colab import files
files.download(recv_file_name)
files.download('receiver_stats.txt')
files.download('sender_stats.txt')

# SEL. REPEAT


In [14]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed

from time import sleep
from random import randint
from threading import Thread, Lock

lock=Lock()

def time_alarm(Packet, prob, time_out, senderSocket,pool,timers):
    time.sleep(time_out)
    
    return Packet, Packet.seq_num, senderSocket, prob, time_out, pool,timers
   
def resend(fn):
    
    packet,number,senderSocket, prob, time_out, pool, timers=fn.result() # fn.result is returned result of time_alarm
    
    if decision(prob):
      with lock:
        try:
          senderSocket.sendto(pickle.dumps(packet),(receiverIP, receiverPort))
          print("resent",number,"actual",packet.act_num)
        except:
          pass
          
          
    else:
        pass
    timers[number]=pool.submit(time_alarm, packet, prob, time_out,senderSocket,pool)

            
            

In [15]:
def is_in_range(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start,start+step_size)]
  if num in lim:
    return True
  else:
    return False
  
def is_in_lrange(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start-1,start-step_size)]
  if num in lim:
    return True
  else:
    return False
    

          

In [16]:
#@title Parameters Tuning{ form-width: "27%" }
sent_file_name = 'Podington_Bear_-_Starling.mp3'  # @param {type:"string"}
recv_file_name = 'sel_rep.mp3'  # @param {type:"string"}
prob = 1  # @param {type:"slider", min:0, max:1, step:0.01}
window_size = 4  # @param {type:"integer"}
seq_num_range = 8  # @param {type:"integer"}
time_out = 0.1 # @param {type:"number"}

import heapq
import os
import time
import threading
from threading import Thread
start_time = time.time()

In [17]:
def recv_child():
    stats_ = {}
    #UDP socket
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    #reuse address when done with it
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))

    print ('--The receiver is ready to receive--\n')

    packets = []
    window = []
    received = []
    recv_base = 0
    flag_end = False
#   receiving while loop, message is a packet of the file sent
    while True:
        (message, senderAddress) = receiverSocket.recvfrom(8192)
        
        message = pickle.loads(message)
        #last packet
        if message.type_ == 0:
            flag_end = True
        #if packet received is within current window
        if is_in_lrange(message.seq_num, recv_base, window_size,
                        seq_num_range) or is_in_range(message.seq_num,
                recv_base, window_size, seq_num_range):
            print("YES in range:",message.act_num)
            if True:
                
                receiverSocket.sendto(pickle.dumps(ACK(message.seq_num,
                        0, message.act_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver sent ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)
                print (stats_['{0:.6f}'.format(t6 - start_time)])
            #else if packet is dropped
            else:
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver dropped ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)

        if message.seq_num in received:
            continue

        if message.seq_num<window_size:
          if (message.seq_num+seq_num_range- window_size) in received:
            received.remove(message.seq_num+seq_num_range- window_size)
        else:
          if (message.seq_num-window_size) in received:
            received.remove(message.seq_num-window_size)

        received.append(message.seq_num)

        if recv_base == message.seq_num:
            window.append(message)
            for (i, j) in zip(sorted(window)[0::1], sorted(window)[1::1]):
                if j.seq_num == (i.seq_num + 1) % seq_num_range \
                    or j.seq_num == i.seq_num + 1:
                    recv_base = j.seq_num
                    packets.append(i)
                    window.remove(i)

                else:
                    recv_base = (i.seq_num + 1) % seq_num_range
                    packets.append(i)
                    window.remove(i)

                    break

            if len(window) == 1 and recv_base == window[0].seq_num:
                recv_base = (window[0].seq_num + 1) % seq_num_range
                packets.append(window[0])
                window.remove(window[0])
                

            
        else:
            window.append(message)
            
        if packets:
          if flag_end:
            time.sleep(0.1)

            print ('--Whole file received successfully. File is ready to download--\n')

            get_stats('receiver_stats.txt', stats_)

            pkt_to_file(recv_file_name, packets)
            receiverSocket.close()
            break

    return



In [18]:
#Sender
def send_parent():
    #sender process
    newpid = os.fork()
    #if in child
    if newpid == 0:
        recv_child()
    else:#parent
        stats_ = {}
        time.sleep(1)
        packets = file_to_pkt(file_name=sent_file_name, sw=False,
                              seq_num_range=seq_num_range)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))

        # ## initial setting of window
        print("length of packets:",len(packets))
        base = 0
        index = 0
        step = 0
        window = []
        timers = {}
        pool = ThreadPoolExecutor(2*window_size)
        not_acked = []
        acked_num = 0
#         t=Thread(target=resend,args=(senderSocket, timers, prob, pool))
#         t.start()
#append and variable change orignal if ERROR
        if index + window_size < len(packets):
            window = packets[index:(index + window_size)]
        else:
            window = packets[index:len(packets)]

        for i in range(len(window)):
            #for each packet in window create an event object and  a timer/thread
            #key of these dics to get these object is packet sequence number
            #senderSocket.sendto(pickle.dumps(window[i]),(receiverIP, receiverPort))
            timers[window[i].seq_num] = pool.submit(time_alarm,window[i],prob,time_out,senderSocket,pool,timers)
            timers[window[i].seq_num].add_done_callback(resend)
            not_acked.append(window[i].seq_num)

        while True:

            reply = None
            (reply, receiverAddress) = senderSocket.recvfrom(4096)
            #time.sleep(0.001)
            reply = pickle.loads(reply)
            print("ack seq_num:",reply.seq_num,reply.real,"REC. ACK")
            if isinstance(reply,ACK):
                #time.sleep(0.0001)
                

                if (reply.seq_num in timers.keys()):
                    del timers[reply.seq_num]
                    try:
                      not_acked.remove(reply.seq_num)
                    except ValueError:
                      print(not_acked,reply.seq_num)

                    acked_num += 1

                    if acked_num == len(packets):
                        senderSocket.close()
                        break

                    if base == reply.seq_num:
                        if len(not_acked)==0:
                            step=window_size
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) > window_size - 1:
                            step = abs(seq_num_range % base - heapq.nsmallest(1, not_acked)[0])
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) <= window_size - 1:
                            step = abs(base - heapq.nsmallest(1, not_acked)[0])

                        index+=step

                        if index + window_size < len(packets):
                            
                            window = packets[index:index + window_size]
 
                        else:

                            window = packets[index:len(packets)]

                        

                        base= window[0].seq_num
                        print(window[0].act_num," ---- ",window[len(window)-1].act_num)

                        for item in window[-step:]:
                            #senderSocket.sendto(pickle.dumps(item),(receiverIP, receiverPort))
                            timers[item.seq_num]= pool.submit(time_alarm,item,prob,time_out,senderSocket,pool,timers)
                            timers[item.seq_num].add_done_callback(resend)
                            not_acked.append(item.seq_num)



        
    return





In [19]:
send_parent()

--The receiver is ready to receive--

YES in range: 1
receiver sent ack for packet 1 actual: 1
YES in range: 0
receiver sent ack for packet 0 actual: 0
YES in range: 2
receiver sent ack for packet 2 actual: 2
YES in range: 3
receiver sent ack for packet 3 actual: 3
length of packets: 4165
resent resentresentresentack seq_num:1     0231actual     actualactualactual11    
023REC. ACK



ack seq_num: 0 0 REC. ACK
2  ----  5
ack seq_num: 2 2 REC. ACK
3  ----  6
ack seq_num: 3 3 REC. ACK
4  ----  7
YES in range: 5
receiver sent ack for packet 5 actual: 5
YES in range: 4
receiver sent ack for packet 4 actual: 4
YES in range: 6
receiver sent ack for packet 6 actual: 6
YES in range: 7
receiver sent ack for packet 7 actual: 7
YES in range: 8
receiver sent ack for packet 0 actual: 8
YES in range: 9
receiver sent ack for packet 1 actual: 9
YES in range: 10
receiver sent ack for packet 2 actual: 10
YES in range: 11
receiver sent ack for packet 3 actual: 11
resentresentack seq_num:resent    5456   

YES in range: 75
receiver sent ack for packet 3 actual: 75
resentack seq_num:  44  actual68  68REC. ACK

69  ----  72
resent resent6 ack seq_num: 5 actual 6 actual 70 70
69 REC. ACK

resentack seq_num:  75  actual69  71REC. ACK

72  ----  75
ack seq_num: 7 71 REC. ACK
resent 0 actual 72
ack seq_num: 0 72 REC. ACK
73  ----  76
resentresent  12  actualactual  7374

resent 3 actual 75
ack seq_num: 1 73 REC. ACK
74  ----  77
ack seq_num: 2 74 REC. ACK
75  ----  78
ack seq_num: 3 75 REC. ACK
76  ----  79
YES in range: 76
receiver sent ack for packet 4 actual: 76
YES in range: 77
receiver sent ack for packet 5 actual: 77
YES in range: 78
receiver sent ack for packet 6 actual: 78
YES in range: 79
receiver sent ack for packet 7 actual: 79
YES in range: 80
receiver sent ack for packet 0 actual: 80
YES in range: 81
receiver sent ack for packet 1 actual: 81
YES in range: 82
receiver sent ack for packet 2 actual: 82
YES in range: 83
receiver sent ack for packet 3 actual: 83
resent 4 actual 76
ack 

YES in range: 142
receiver sent ack for packet 6 actual: 142
YES in range: 144
receiver sent ack for packet 0 actual: 144
YES in range: 145
receiver sent ack for packet 1 actual: 145
YES in range: 146
receiver sent ack for packet 2 actual: 146
YES in range: 147
receiver sent ack for packet 3 actual: 147
resent 4 actual 140
ack seq_num: 4 140 REC. ACK
141  ----  144
resent 5 actual 141
ack seq_num: 5 resent141 REC. ACK
144  ----   resent1477 
 6actual  actual143 
142
ack seq_num: 7 143 REC. ACK
ack seq_num: 6 142 REC. ACK
resent 0 actual 144
ack seq_num: 0 144 REC. ACK
145  ----  148
resent 1 actual 145
resent 2ack seq_num:resent   actual13   146145actual
  147REC. ACK

146  ----  149
ack seq_num: 2 146 REC. ACK
147  ----  150
ack seq_num: 3 147 REC. ACK
148  ----  151
YES in range: 148
receiver sent ack for packet 4 actual: 148
YES in range: 149
receiver sent ack for packet 5 actual: 149
YES in range: 150
receiver sent ack for packet 6 actual: 150
YES in range: 151
receiver sent ack fo

YES in range: 212
receiver sent ack for packet 4 actual: 212
YES in range: 213
receiver sent ack for packet 5 actual: 213
YES in range: 214
receiver sent ack for packet 6 actual: 214
YES in range: 215
receiver sent ack for packet 7 actual: 215
YES in range: 216
receiver sent ack for packet 0 actual: 216
YES in range: 217
receiver sent ack for packet 1 actual: 217
YES in range: 219
receiver sent ack for packet 3 actual: 219
YES in range: 218
receiver sent ack for packet 2 actual: 218
resent 4 actual 212
ack seq_num: 4 212 REC. ACK
213  ----  216
resent 5ack seq_num:resent   actual56   213213actual
  214REC. ACK

216  ----  219
ack seq_num: 6 214 REC. ACK
resent 7ack seq_num:  actual7  215215
 REC. ACK
resent 0 actual 216
ack seq_num: 0 216 REC. ACK
217  ----  220
resentresentresent   132   actualactualactual   217219218


ack seq_num: 1 217 REC. ACK
218  ----  221
ack seq_num: 3 219 REC. ACK
ack seq_num: 2 218 REC. ACK
220  ----  223
YES in range: 220
receiver sent ack for packet 4 actu

YES in range: 284
receiver sent ack for packet 4 actual: 284
YES in range: 285
receiver sent ack for packet 5 actual: 285
YES in range: 286
receiver sent ack for packet 6 actual: 286
YES in range: 287
receiver sent ack for packet 7 actual: 287
YES in range: 288
receiver sent ack for packet 0 actual: 288
YES in range: 289
receiver sent ack for packet 1 actual: 289
YES in range: 290
receiver sent ack for packet 2 actual: 290
YES in range: 291
receiver sent ack for packet 3 actual: 291
resent 4 actual 284
ack seq_num: 4 284 REC. ACK
285  ----  288
resent 5 actual 285
resent 6 actual 286
ack seq_num: 5 285 REC. ACK
288  ----  291
ack seq_num: 6 286 REC. ACK
resent 7 actual 287
ack seq_num: 7 287 REC. ACK
resent 0 actual 288
ack seq_num: 0 288 REC. ACK
289  ----  292
resentresent  resentack seq_num:12    31actualactual    actual289289290  

291REC. ACK

290  ----  293
ack seq_num: 2 290 REC. ACK
291  ----  294
ack seq_num: 3 291 REC. ACK
292  ----  295
YES in range: 292
receiver sent ack fo

...

YES in range: 3868
receiver sent ack for packet 4 actual: 3868
YES in range: 3870
receiver sent ack for packet 6 actual: 3870
YES in range: 3869
receiver sent ack for packet 5 actual: 3869
YES in range: 3871
receiver sent ack for packet 7 actual: 3871
YES in range: 3872
receiver sent ack for packet 0 actual: 3872
resent resent2  1actual  actual3866 ack seq_num:resent
3865  3
2  actual3866  3867REC. ACK

ack seq_num: 1 3865 REC. ACK
3867  ----  3870
ack seq_num: 3 3867 REC. ACK
3868  ----  3871
resent 4 actual 3868
ack seq_num: 4 3868 REC. ACK
3869  ----  3872
resent 6 actual 3870
resent 5 actual 3869
ack seq_num: 6 3870resent 7 actual 3871
 REC. ACK
ack seq_num: 5 3869 REC. ACK
3872  ----  3875
ack seq_num: 7 3871 REC. ACK
resent 0 actual 3872
ack seq_num: 0 3872 REC. ACK
3873  ----  3876
YES in range: 3874
receiver sent ack for packet 2 actual: 3874
YES in range: 3875
receiver sent ack for packet 3 actual: 3875
YES in range: 3873
receiver sent ack for packet 1 actual: 3873
YES in rang

YES in range: 3936
receiver sent ack for packet 0 actual: 3936
resent 1 actual ack seq_num:resent3929  
12  actual3929  3930REC. ACK

3930  ----  3933
ack seq_num: 2 3930 REC. ACK
3931  ----  3934
resent 4 actual 3932ack seq_num: 4 3932 REC. ACK

resent 3 actual 3931
ack seq_num: 3 3931 REC. ACK
3933  ----  3936
resent resent5  6actual  actual3933 
3934
ack seq_num: 5 3933 REC. ACK
3936  ----  3939
ack seq_num: 6 3934 REC. ACK
resentresent  70  ack seq_num:actualactual   739353936 

3935 REC. ACK
ack seq_num: 0 3936 REC. ACK
3937  ----  3940
YES in range: 3937
receiver sent ack for packet 1 actual: 3937
YES in range: 3938
receiver sent ack for packet 2 actual: 3938
YES in range: 3939
receiver sent ack for packet 3 actual: 3939
YES in range: 3940
receiver sent ack for packet 4 actual: 3940
YES in range: 3941
receiver sent ack for packet 5 actual: 3941
YES in range: 3942
receiver sent ack for packet 6 actual: 3942
YES in range: 3943
receiver sent ack for packet 7 actual: 3943
YES in rang

YES in range: 4001
receiver sent ack for packet 1 actual: 4001
YES in range: 4002
receiver sent ack for packet 2 actual: 4002
YES in range: 4003
receiver sent ack for packet 3 actual: 4003
YES in range: 4004
receiver sent ack for packet 4 actual: 4004
YES in range: 4005
receiver sent ack for packet 5 actual: 4005
YES in range: 4006
receiver sent ack for packet 6 actual: 4006
YES in range: 4007
receiver sent ack for packet 7 actual: 4007
YES in range: 4008
receiver sent ack for packet 0 actual: 4008
resent 1 resentack seq_num:actual  resent 21 4001  3
actual 4001  actualREC. ACK4002 

40034002
  ----  4005
ack seq_num: 2 4002 REC. ACK
4003  ----  4006
ack seq_num: 3 4003 REC. ACK
4004  ----  4007
resent 4 actual 4004
ack seq_num: 4 4004 REC. ACK
4005  ----  4008
resent 5 actual 4005
ack seq_num: 5 4005 REC. ACK
4008  ----  4011
resent 6 actual 4006
resentack seq_num:  76  actual4006  4007REC. ACK

ack seq_num: 7 4007 REC. ACK
resent 0 actual 4008
ack seq_num: 0 4008 REC. ACK
4009  ---- 

YES in range: 4068
receiver sent ack for packet 4 actual: 4068
YES in range: 4069
receiver sent ack for packet 5 actual: 4069
YES in range: 4070
receiver sent ack for packet 6 actual: 4070
YES in range: 4071
receiver sent ack for packet 7 actual: 4071
YES in range: 4072
receiver sent ack for packet 0 actual: 4072
resentresent  12resentack seq_num:    actualactual31    40654066actual4065

  REC. ACK4067

4066  ----  4069
ack seq_num: 2 4066 REC. ACK
4067  ----  4070
ack seq_num: 3 4067 REC. ACK
4068  ----  4071
resent 4 actual 4068
ack seq_num: 4 4068 REC. ACK
4069  ----  4072
resent 5 actual 4069
ack seq_num: 5 4069 REC. ACK
4072  ----  4075
resent 6 actual 4070
resentack seq_num:  67  4070actual  REC. ACK4071

ack seq_num: 7 4071 REC. ACK
resent 0 actual 4072
ack seq_num: 0 4072 REC. ACK
4073  ----  4076
YES in range: 4074
receiver sent ack for packet 2 actual: 4074
YES in range: 4073
receiver sent ack for packet 1 actual: 4073
YES in range: 4075
receiver sent ack for packet 3 actual:

YES in range: 4136
receiver sent ack for packet 0 actual: 4136
resentresent resentack seq_num: 2  1 31 actual  actual actual4129 4130  4129
4131REC. ACK


4130  ----  4133
ack seq_num: 2 4130 REC. ACK
4131  ----  4134
ack seq_num: 3 4131 REC. ACK
4132  ----  4135
resent 4 actual 4132
ack seq_num: 4 4132 REC. ACK
4133  ----  4136
resent 5 actual 4133
ack seq_num: 5 4133 REC. ACK
4136  ----  4139
resent 6 actual 4134
ack seq_num: 6 4134 REC. ACK
resent 7 actual 4135
ack seq_num: 7 4135 REC. ACK
resent 0 actual 4136
ack seq_num: 0 4136 REC. ACK
4137  ----  4140
YES in range: 4137
receiver sent ack for packet 1 actual: 4137
YES in range: 4138
receiver sent ack for packet 2 actual: 4138
YES in range: 4139
receiver sent ack for packet 3 actual: 4139
YES in range: 4140
receiver sent ack for packet 4 actual: 4140
YES in range: 4141
receiver sent ack for packet 5 actual: 4141
YES in range: 4142
receiver sent ack for packet 6 actual: 4142
YES in range: 4143
receiver sent ack for packet 7 actual:

In [ ]:
from google.colab import files
files.download(recv_file_name)